In [1]:
#| default_exp py2pyi

# Create delegated pyi

## Setup

In [2]:
#| export
import ast, sys, inspect, re, os, importlib.util, importlib.machinery

from ast import parse, unparse
from inspect import signature, getsource
from fastcore.utils import *
from fastcore.meta import delegates
from pprint import pprint

## Basics

In [3]:
#| export
def imp_mod(module_path, package=None):
    "Import dynamically the module referenced in `fn`"
    module_path = str(module_path)
    module_name = os.path.splitext(os.path.basename(module_path))[0]
    spec = importlib.machinery.ModuleSpec(module_name, None, origin=module_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader = importlib.machinery.SourceFileLoader(module_name, module_path)
    if package is not None: module.__package__ = package
    module.__file__ = os.path.abspath(module_path)
    spec.loader.exec_module(module)
    return module

In [4]:
fn = Path('test_py2pyi.py')

In [5]:
mod = imp_mod(fn)
a = mod.A()
a.h()

1

In [6]:
#| export
def _get_tree(mod):
    return parse(getsource(mod))

In [7]:
tree = _get_tree(mod)

In [8]:
tree


In [9]:
print(unparse(tree)) # now I know what exactly is tree, and tree is from AST ✨✨✨✨

__all__ = ['f', 'g']
from fastcore.meta import delegates
from fastcore.utils import patch

class X(int):
    pass

def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1

@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2

def j(c: int, d: str='a') -> str:
    """I am j"""
    return 1

class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a

class B:
    ...

@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1

@patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1

@patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    return 1


In [10]:
#| export
@patch
def __repr__(self:ast.AST):
    return unparse(self)

@patch
def _repr_markdown_(self:ast.AST):
    return f"""```python
{self!r}
```"""

In [11]:
for o in enumerate(tree.body): print(o) # now I know what inside tree.body, and maybe each item is a node? ✨✨✨✨

(0, __all__ = ['f', 'g'])
(1, from fastcore.meta import delegates)
(2, from fastcore.utils import patch)
(3, class X(int):
    pass)
(4, def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1)
(5, @delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2)
(6, def j(c: int, d: str='a') -> str:
    """I am j"""
    return 1)
(7, class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a)
(8, class B:
    ...)
(9, @patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1)
(10, @patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1)
(11, @patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    return 1)


In [20]:
node_func = tree.body[4] # yes, Jeremy make each item as a node ✨✨✨✨
node_func

```python
def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1
```

In [21]:
#| export
functypes = (ast.FunctionDef,ast.AsyncFunctionDef) # consider both normal func and async func as functypes

In [22]:
isinstance(node_func, functypes)

True

In [15]:
#| export
def _deco_id(d:Union[ast.Name,ast.Attribute])->bool:
    "Get the id for AST node `d`"
    return d.id if isinstance(d, ast.Name) else d.func.ids

def has_deco(node:Union[ast.FunctionDef,ast.AsyncFunctionDef], name:str)->bool:
    "Check if a function node `node` has a decorator named `name`"
    return any(_deco_id(d)==name for d in getattr(node, 'decorator_list', []))

In [23]:
nm = 'delegates'
has_deco(node_func, nm) 
# has_deco(node_pa, nm)# has_deco(node_pa, nm)

False

In [24]:
node_del = tree.body[5]
node_del

```python
@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2
```

In [25]:
has_deco(node_del, 'delegates')

AttributeError: 'Name' object has no attribute 'ids'

## Function processing

In [17]:
def _proc_body   (node, mod): print('_proc_body', type(node))
def _proc_func   (node, mod): print('_proc_func', type(node))
def _proc_class  (node, mod): print('_proc_class', type(node))
def _proc_patched(node, mod): print('_proc_patched', type(node))

In [18]:
#| export
def _get_proc(node):
    if isinstance(node, ast.ClassDef): return _proc_class
    if not isinstance(node, functypes): return None
    if not has_deco(node, 'delegates'): return _proc_body
    if has_deco(node, 'patch'): return _proc_patched
    return _proc_func

In [19]:
#| export
def _proc_tree(tree, mod):
    for node in tree.body:
        proc = _get_proc(node)
        if proc: proc(node, mod)

In [20]:
#| export
def _proc_mod(mod):
       tree = _get_tree(mod)
       _proc_tree(tree, mod)
       # Remove standalone patched functions
       tree.body = [node for node in tree.body 
                    if not (isinstance(node, ast.FunctionDef) and has_deco(node, 'patch'))]
       return tree


# def _proc_mod(mod):
#     tree = _get_tree(mod)
#     _proc_tree(tree, mod)
#     return tree

In [21]:
_proc_mod(mod);

_proc_class <class 'ast.ClassDef'>
_proc_body <class 'ast.FunctionDef'>
_proc_func <class 'ast.FunctionDef'>
_proc_body <class 'ast.FunctionDef'>
_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_body <class 'ast.FunctionDef'>


In [22]:
node.name

'g'

In [23]:
sym = getattr(mod, node.name)
sym

<function test_py2pyi.g(c, d: test_py2pyi.X, *, b: str = 'a') -> str>

In [24]:
sig = signature(sym)
print(sig)

(c, d: test_py2pyi.X, *, b: str = 'a') -> str


In [25]:
#| export
def sig2str(sig):
    s = str(sig)
    s = re.sub(r"<class '(.*?)'>", r'\1', s)
    s = re.sub(r"dynamic_module\.", "", s)
    return s

In [26]:
#| export
def ast_args(func):
    sig = signature(func)
    return ast.parse(f"def _{sig2str(sig)}: ...").body[0].args

In [27]:
newargs = ast_args(sym)
newargs

```python
c, d: test_py2pyi.X, *, b: str='a'
```

In [28]:
node.args

```python
c, d: X, **kwargs
```

In [29]:
node.args = newargs
node

```python
@delegates(f)
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    return 2
```

In [30]:
#| export
def _body_ellip(n: ast.AST):
    stidx = 1 if isinstance(n.body[0], ast.Expr) and isinstance(n.body[0].value, ast.Str) else 0
    n.body[stidx:] = [ast.Expr(ast.Constant(...))]

In [31]:
_body_ellip(node)
node

```python
@delegates(f)
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    ...
```

In [32]:
#| export
def _update_func(node, sym):
    """Replace the parameter list of the source code of a function `f` with a different signature.
    Replace the body of the function with just `pass`, and remove any decorators named 'delegates'"""
    node.args = ast_args(sym)
    _body_ellip(node)
    node.decorator_list = [d for d in node.decorator_list if _deco_id(d) != 'delegates']

In [33]:
tree = _get_tree(mod)
node = tree.body[5]
node

```python
@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2
```

In [34]:
_update_func(node, sym)
node

```python
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    ...
```

In [35]:
#| export
def _proc_body(node, mod): _body_ellip(node)

In [36]:
#| export
def _proc_func(node, mod):
    sym = getattr(mod, node.name)
    _update_func(node, sym)

In [37]:
tree = _proc_mod(mod)
tree.body[5]

_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_patched <class 'ast.FunctionDef'>
_proc_patched <class 'ast.FunctionDef'>


```python
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    ...
```

## Patch

In [42]:
# node = tree.body[8]
node = tree.body[9]
node

```python
class B:
    ...
```

In [43]:
ann = node.args.args[0].annotation

AttributeError: 'ClassDef' object has no attribute 'args'

In [ ]:
if hasattr(ann, 'elts'): ann = ann.elts[0]

In [ ]:
nm = ann.id
nm

'A'

In [ ]:
cls = getattr(mod, nm)
sym = getattr(cls, node.name)

In [ ]:
sig2str(signature(sym))

"(self: (test_py2pyi.A, test_py2pyi.B), b: bool = False, *, d: str = 'a')"

In [ ]:
_update_func(node, sym)

In [ ]:
node

```python
@patch
def k(self: (test_py2pyi.A, test_py2pyi.B), b: bool=False, *, d: str='a'):
    ...
```

In [44]:
#| export
def _proc_patched(node, mod): pass

# def _proc_patched(node, mod):
#     ann = node.args.args[0].annotation
#     if hasattr(ann, 'elts'): ann = ann.elts[0]
#     cls = getattr(mod, ann.id)
#     sym = getattr(cls, node.name)
#     _update_func(node, sym)

In [45]:
tree = _proc_mod(mod)
tree.body[9]

_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>
_proc_class <class 'ast.ClassDef'>


IndexError: list index out of range

## Class and file

In [ ]:
tree = _get_tree(mod)
node = tree.body[7]
node

```python
class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a
```

In [ ]:
node.body

[@delegates(j)
 def h(self, b: bool=False, **kwargs):
     a = 1
     return a]

In [46]:
def _add_method_to_class(class_node, method_name, method_object):
       method_ast = ast.FunctionDef(
           name=method_name,
           args=ast_args(method_object),
           body=[ast.Expr(ast.Constant(...))],
           decorator_list=[]
       )
       class_node.body.append(method_ast) # presumably, class_node is from AST, method_object should be from runtime, and method_ast is a new AST node ✨✨✨✨✨

In [47]:
#| export
def _proc_class(node, mod):
       cls = getattr(mod, node.name) # node from AST and mod from runtime? what is cls from? ✨✨✨✨✨
       for name, member in cls.__dict__.items():
           if callable(member) and not name.startswith('__'):
               # This could be a patched method
               if name not in [n.name for n in node.body if isinstance(n, ast.FunctionDef)]:
                   # It's not in the original class body, so it's likely patched
                   _add_method_to_class(node, name, member) # presumably, node is from AST, name and member are from runtime ✨✨✨✨✨
       _proc_tree(node, cls)

# def _proc_class(node, mod):
#     cls = getattr(mod, node.name)
#     _proc_tree(node, cls)

In [48]:
tree = _proc_mod(mod)
tree.body[7]

AttributeError: 'FunctionDef' object has no attribute 'lineno'

AttributeError: 'FunctionDef' object has no attribute 'lineno'

In [49]:
#| export
def create_pyi(fn, package=None):
    "Convert `fname.py` to `fname.pyi` by removing function bodies and expanding `delegates` kwargs"
    fn = Path(fn)
    mod = imp_mod(fn, package=package)
    tree = _proc_mod(mod)
    res = unparse(tree)
    fn.with_suffix('.pyi').write_text(res)

In [50]:
fn = 'simple-test.py'
create_pyi(fn)

AttributeError: 'FunctionDef' object has no attribute 'lineno'

In [55]:
# fn = Path('/Users/jhoward/git/fastcore/fastcore/docments.py')
# create_pyi(fn, 'fastcore')

## Script

In [56]:
#| export
from fastcore.script import call_parse

In [57]:
#| export
@call_parse
def py2pyi(fname:str,  # The file name to convert
           package:str=None  # The parent package
          ):
    "Convert `fname.py` to `fname.pyi` by removing function bodies and expanding `delegates` kwargs"
    create_pyi(fname, package)

In [58]:
#| export
@call_parse
def replace_wildcards(
    # Path to the Python file to process
    path: str):
    "Expand wildcard imports in the specified Python file."
    path = Path(path)
    path.write_text(expand_wildcards(path.read_text()))

# Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

# simple-test.py experiment

In [61]:
from pprint import pprint
fn = 'simple-test.py'

# output: Path('simple-test.py')

def create_pyi(fn, package=None):
    "Convert `fname.py` to `fname.pyi` by removing function bodies and expanding `delegates` kwargs"
    fn = Path(fn)

    mod = imp_mod(fn, package=None)
    tree = _proc_mod(mod)
    res = unparse(tree)
    fn.with_suffix('.pyi').write_text(res)


def imp_mod(module_path, package=None):
    "Import dynamically the module referenced in `fn`"
    module_path = str(module_path)
    print("module_path: ", module_path)
    module_name = os.path.splitext(os.path.basename(module_path))[0]
    print("module_name: ", module_name)
    spec = importlib.machinery.ModuleSpec(module_name, None, origin=module_path)
    print("spec: ", spec)
    module = importlib.util.module_from_spec(spec)
    print("module: ", module)
    spec.loader = importlib.machinery.SourceFileLoader(module_name, module_path)
    print("spec.loader: ", spec.loader)

    if package is not None: module.__package__ = package
    module.__file__ = os.path.abspath(module_path)
    spec.loader.exec_module(module)
    return module

mod = imp_mod(fn, package=None)
pprint(mod)


module_path:  simple-test.py
module_name:  simple-test
spec:  ModuleSpec(name='simple-test', loader=None, origin='simple-test.py')
module:  <module 'simple-test' (simple-test.py)>
spec.loader:  <_frozen_importlib_external.SourceFileLoader object at 0x11f56a1a0>
<module 'simple-test' (simple-test.py)>


In [60]:
#### commented without changes

from pprint import pprint
fn = 'simple-test.py'
fn = Path(fn)
fn
# output: Path('simple-test.py')


def imp_mod(module_path, package=None):
    "Import dynamically the module referenced in `fn`"
    module_path = str(module_path)
    print("module_path: ", module_path)
    module_name = os.path.splitext(os.path.basename(module_path))[0]
    print("module_name: ", module_name)
    spec = importlib.machinery.ModuleSpec(module_name, None, origin=module_path)
    print("spec: ", spec)
    module = importlib.util.module_from_spec(spec)
    print("module: ", module)
    spec.loader = importlib.machinery.SourceFileLoader(module_name, module_path)
    print("spec.loader: ", spec.loader)
    if package is not None: module.__package__ = package
    module.__file__ = os.path.abspath(module_path)
    spec.loader.exec_module(module)
    return module

mod = imp_mod(fn, package=None)
pprint(mod)

#| export
# Function to process a module
def _proc_mod(mod):
    # Parse the module's source code into an abstract syntax tree (AST)
    tree = _get_tree(mod)
    # Process the AST
    _proc_tree(tree, mod)
    # Return the processed AST
    return tree

# Function to get the AST of a module
def _get_tree(mod):
    # Parse the source code of the module into an AST and return it
    return parse(getsource(mod))

# Function to process an AST
def _proc_tree(tree, mod):
    # Iterate over each node in the body of the tree
    for node in tree.body:
        # Get a processing function for the node
        proc = _get_proc(node)
        # If a processing function was returned, call it with the node and the module
        if proc: proc(node, mod)

# Function to process a class node
def _proc_class(node, mod):
    # Get the class object from the module
    cls = getattr(mod, node.name)
    # Process the class's AST
    _proc_tree(node, cls)

# Function to process a function or method node that doesn't have a 'delegates' decorator
def _proc_body(node, mod): 
    # Replace the body of the function or method with an ellipsis
    _body_ellip(node)

# Function to process a function or method node that has a 'patch' decorator
def _proc_patched(node, mod):
    # Get the annotation of the first argument of the function or method
    ann = node.args.args[0].annotation
    # If the annotation has elements, get the first one
    if hasattr(ann, 'elts'): ann = ann.elts[0]
    # Get the class object from the module
    cls = getattr(mod, ann.id)
    # Get the symbol (function or method) from the class
    sym = getattr(cls, node.name)
    # Update the function or method node with the symbol's signature and remove any 'delegates' decorators
    _update_func(node, sym)

# Function to get the id of a decorator
def _deco_id(d:Union[ast.Name,ast.Attribute])->bool:
    # If the decorator is a name, return its id
    # If the decorator is an attribute, return the id of its function
    return d.id if isinstance(d, ast.Name) else d.func.id

# Function to check if a function or method node has a specific decorator
def has_deco(node:Union[ast.FunctionDef,ast.AsyncFunctionDef], name:str)->bool:
    # Return True if any of the node's decorators have the specified name, False otherwise
    return any(_deco_id(d)==name for d in getattr(node, 'decorator_list', []))

# Function to update a function or method node
def _update_func(node, sym):
    # Replace the node's argument list with the symbol's signature
    node.args = ast_args(sym)
    # Replace the body of the node with an ellipsis
    _body_ellip(node)
    # Remove any 'delegates' decorators from the node
    node.decorator_list = [d for d in node.decorator_list if _deco_id(d) != 'delegates']

#| export
# Function to get a processing function for a node
def _get_proc(node):
    # If the node is a class definition, return the class processing function
    if isinstance(node, ast.ClassDef): return _proc_class
    # If the node is not a function or method definition, return None
    if not isinstance(node, functypes): return None
    # If the node doesn't have a 'delegates' decorator, return the body processing function
    if not has_deco(node, 'delegates'): return _proc_body
    # If the node has a 'patch' decorator, return the patched processing function
    if has_deco(node, 'patch'): return _proc_patched
    # Otherwise, return the function processing function
    return _proc_func

In [ ]:
# Function to process a module
def _proc_mod(mod):
    print(f"Processing module: {mod}")
    tree = _get_tree(mod)
    print(f"AST of module: {tree}")
    _proc_tree(tree, mod)
    return tree

def _get_tree(mod):
    print(f"Getting AST for module: {mod}")
    return parse(getsource(mod))

def _proc_tree(tree, mod):
    print(f"Processing AST: {tree} for module: {mod}")
    for node in tree.body:
        proc = _get_proc(node)
        if proc: 
            print(f"Processing node: {node} with function: {proc}")
            proc(node, mod)

def _proc_class(node, mod):
    print(f"Processing class node: {node} in module: {mod}")
    cls = getattr(mod, node.name)
    _proc_tree(node, cls)

def _proc_body(node, mod): 
    print(f"Processing function/method node without 'delegates' decorator: {node} in module: {mod}")
    _body_ellip(node)

def _proc_patched(node, mod):
    print(f"Processing function/method node with 'patch' decorator: {node} in module: {mod}")
    ann = node.args.args[0].annotation
    if hasattr(ann, 'elts'): ann = ann.elts[0]
    cls = getattr(mod, ann.id)
    sym = getattr(cls, node.name)
    _update_func(node, sym)

def _deco_id(d:Union[ast.Name,ast.Attribute])->bool:
    print(f"Getting id for decorator: {d}")
    return d.id if isinstance(d, ast.Name) else d.func.id

def has_deco(node:Union[ast.FunctionDef,ast.AsyncFunctionDef], name:str)->bool:
    print(f"Checking if node: {node} has decorator: {name}")
    return any(_deco_id(d)==name for d in getattr(node, 'decorator_list', []))

def _update_func(node, sym):
    print(f"Updating function/method node: {node} with symbol: {sym}")
    node.args = ast_args(sym)
    _body_ellip(node)
    node.decorator_list = [d for d in node.decorator_list if _deco_id(d) != 'delegates']

def _get_proc(node):
    print(f"Getting processing function for node: {node}")
    if isinstance(node, ast.ClassDef): return _proc_class
    if not isinstance(node, functypes): return None
    if not has_deco(node, 'delegates'): return _proc_body
    if has_deco(node, 'patch'): return _proc_patched
    return _proc_func


In [67]:
%%debug
tree = _proc_mod(mod)
pprint(tree)


In [64]:
# #### changed solution

# # Function to process a module
# def _proc_mod(mod):
#     # Parse the module's source code into an abstract syntax tree (AST)
#     tree = _get_tree(mod)
#     # Process the AST
#     _proc_tree(tree, mod)
#     # Return the processed AST
#     return tree

# # Function to get the AST of a module
# def _get_tree(mod):
#     # Parse the source code of the module into an AST and return it
#     return parse(getsource(mod))

# # Function to process an AST
# def _proc_tree(tree, mod):
#     # Iterate over each node in the body of the tree
#     for node in tree.body:
#         # Get a processing function for the node
#         proc = _get_proc(node)
#         # If a processing function was returned, call it with the node and the module
#         if proc: proc(node, mod)

#     # Move patched functions into their respective classes
#     for node in tree.body:
#         if isinstance(node, ast.FunctionDef) and has_deco(node, 'patch'):
#             _proc_patched(node, mod)
#             tree.body.remove(node)

# # Function to process a class node
# def _proc_class(node, mod):
#     # Get the class object from the module
#     cls = getattr(mod, node.name)
#     # Process the class's AST
#     _proc_tree(node, cls)

# # Function to process a function or method node that doesn't have a 'delegates' decorator
# def _proc_body(node, mod): 
#     # Replace the body of the function or method with an ellipsis
#     _body_ellip(node)

# # Function to process a function or method node that has a 'patch' decorator
# def _proc_patched(node, mod):
#     # Get the annotation of the first argument of the function or method
#     ann = node.args.args[0].annotation
#     # If the annotation has elements, get the first one
#     if hasattr(ann, 'elts'): ann = ann.elts[0]
#     # Get the class object from the module
#     cls = getattr(mod, ann.id)
#     # Get the symbol (function or method) from the class
#     sym = getattr(cls, node.name)
#     # Update the function or method node with the symbol's signature and remove any 'delegates' decorators
#     _update_func(node, sym)

#     # Remove the function from the module level
#     mod.__dict__.pop(node.name, None)
#     # Add the function to the class level
#     setattr(cls, node.name, node)

#     # Find the class node in the AST
#     for class_node in tree.body:
#         if isinstance(class_node, ast.ClassDef) and class_node.name == cls.__name__:
#             # Add the function node to the body of the class node
#             class_node.body.append(node)
#             break

# # Function to get the id of a decorator
# def _deco_id(d:Union[ast.Name,ast.Attribute])->bool:
#     # If the decorator is a name, return its id
#     # If the decorator is an attribute, return the id of its function
#     return d.id if isinstance(d, ast.Name) else d.func.id

# # Function to check if a function or method node has a specific decorator
# def has_deco(node:Union[ast.FunctionDef,ast.AsyncFunctionDef], name:str)->bool:
#     # Return True if any of the node's decorators have the specified name, False otherwise
#     return any(_deco_id(d)==name for d in getattr(node, 'decorator_list', []))

# # Function to update a function or method node
# def _update_func(node, sym):
#     # Replace the node's argument list with the symbol's signature
#     node.args = ast_args(sym)
#     # Replace the body of the node with an ellipsis
#     _body_ellip(node)
#     # Remove any 'delegates' decorators from the node
#     node.decorator_list = [d for d in node.decorator_list if _deco_id(d) != 'delegates']

# #| export
# # Function to get a processing function for a node
# def _get_proc(node):
#     # If the node is a class definition, return the class processing function
#     if isinstance(node, ast.ClassDef): return _proc_class
#     # If the node is not a function or method definition, return None
#     if not isinstance(node, functypes): return None
#     # If the node doesn't have a 'delegates' decorator, return the body processing function
#     if not has_deco(node, 'delegates'): return _proc_body
#     # If the node has a 'patch' decorator, return the patched processing function
#     if has_deco(node, 'patch'): return _proc_patched
#     # Otherwise, return the function processing function
#     return _proc_func

In [65]:
tree = _proc_mod(mod)
pprint(tree)


TypeError: @patch
def f(self: simple - test.A):
    ... is not a callable object

In [63]:
res = unparse(tree)